<a href="https://www.kaggle.com/code/chongyockeng/ie4483?scriptVersionId=199848884" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,random_split

import torchvision
from torchvision import datasets, transforms
import torchvision.models as models
from torchvision.transforms import InterpolationMode
from torchinfo import summary

from timeit import default_timer as timer 
from tqdm.auto import tqdm

import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.append('/kaggle/input/custom-packages/')
import data_setup, engine, model, utils

from utils import CLAHETransform, create_writer, save_model
from data_setup import create_dataloaders
from model import create_vgg11, create_vgg16, create_vgg19

In [3]:
config = {"BATCH_SIZE":32,
         "LEARNING_RATE":1e-4}

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
base_dir = "/kaggle/input/final-project-dataset/datasets"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
test_dir = os.path.join(base_dir, "test")

print("Train Directory:", train_dir)
print("Validation Directory:", val_dir)
print("Test Directory:", test_dir)

Train Directory: /kaggle/input/final-project-dataset/datasets/train
Validation Directory: /kaggle/input/final-project-dataset/datasets/val
Test Directory: /kaggle/input/final-project-dataset/datasets/test


In [6]:

# Case 1: VGG Default Transform
transform_case_1 = torchvision.models.VGG19_Weights.DEFAULT.transforms()

# Case 2: CLAHE
transform_case_2 = transforms.Compose([
    CLAHETransform(clip_limit=2.0, tile_grid_size=(8, 8)),  # Apply CLAHE
    transforms.Resize((256, 256), # Resize to 256x256
    interpolation=InterpolationMode.BICUBIC),  
    transforms.CenterCrop(224),  # Crop to 224x224
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Case 3: Rotation + Horizontal Flip
transform_case_3 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 

])
# Case 4: CLAHE + Rotation + Horizontal Flip
transform_case_4 = transforms.Compose([
    CLAHETransform(clip_limit=2.0, tile_grid_size=(8, 8)),  # Apply CLAHE
    transforms.Resize((256, 256), # Resize to 256x256
    interpolation=InterpolationMode.BICUBIC),  
    transforms.CenterCrop(224),  # Crop to 224x224
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
def create_dataloaders_helper(transformation):
    return create_dataloaders(train_dir=train_dir,val_dir=val_dir,transform=transformation,
                              batch_size=config['BATCH_SIZE'])

In [8]:
experiment_dataloaders = {
                        "Default Transform": create_dataloaders_helper(transform_case_1),
                        "CLAHE Transform": create_dataloaders_helper(transform_case_2),
                        "Rotation + Horizontal Flip Transform": create_dataloaders_helper(transform_case_3),
                        "CLAHE + Rotation + Horizontal Flip Transform": create_dataloaders_helper(transform_case_4),
                        }

In [9]:
# 1. Set the random seeds
torch.manual_seed(42)

models = ["vgg11","vgg16","vgg19"]

num_epochs = [5,10]
# 2. Keep track of experiment numbers
experiment_number = 0

# 3. Loop through each DataLoader
for dataloader_name, dataloaders in experiment_dataloaders.items():
    class_names = dataloaders[-1]
    # 4. Loop through each number of epochs
    for epochs in num_epochs: 

        # 5. Loop through each model name and create a new model based on the name
        for model_name in models:

            # 6. Create information print outs
            experiment_number += 1
            print(f"[INFO] Experiment number: {experiment_number}")
            print(f"[INFO] Model: {model_name}")
            print(f"[INFO] DataLoader: {dataloader_name}")
            print(f"[INFO] Number of epochs: {epochs}")  

            # 7. Select the model
            if model_name == "vgg11":
                model = create_vgg11(device=device,class_names=class_names) 
            elif model_name == "vgg16":
                model = create_vgg16(device=device,class_names=class_names)
            else:
                model = create_vgg19(device=device,class_names=class_names) 
            
            # 8. Create a new loss and optimizer for every model
            loss_fn = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(params=model.parameters(), lr=config["LEARNING_RATE"])

            # 9. Train target model with target dataloaders and track experiments
            engine.train(model=model,
                  train_dataloader=dataloaders[0],
                  test_dataloader=dataloaders[1], 
                  val_dataloader=dataloaders[2],
                  optimizer=optimizer,
                  loss_fn=loss_fn,
                  epochs=epochs,
                  device=device,
                  writer=create_writer(experiment_name=dataloader_name,
                                       model_name=model_name,
                                       extra=f"{epochs}_epochs"))
            
            # 10. Save the model to file so we can get back the best model
            save_filepath = f"{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            save_model(model=model,
                       target_dir="models",
                       model_name=save_filepath)
            print("-"*50 + "\n")

[INFO] Experiment number: 1
[INFO] Model: vgg11
[INFO] DataLoader: Default Transform
[INFO] Number of epochs: 5


Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth
100%|██████████| 507M/507M [00:05<00:00, 93.3MB/s]


[INFO] Created new VGG11 model.
[INFO] Created SummaryWriter, saving to: logs/2024-10-07/Default Transform/vgg11/5_epochs...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.1459 | train_acc: 0.9584 | test_loss: 0.1301 | test_acc: 0.9653
Epoch: 2 | train_loss: 0.0395 | train_acc: 0.9871 | test_loss: 0.0810 | test_acc: 0.9792
Epoch: 3 | train_loss: 0.0246 | train_acc: 0.9934 | test_loss: 0.0726 | test_acc: 0.9832
Epoch: 4 | train_loss: 0.0207 | train_acc: 0.9937 | test_loss: 0.1135 | test_acc: 0.9760
Epoch: 5 | train_loss: 0.0238 | train_acc: 0.9940 | test_loss: 0.1193 | test_acc: 0.9810
[INFO] Saving model to: models/vgg11_Default Transform_5_epochs.pth
--------------------------------------------------

[INFO] Experiment number: 2
[INFO] Model: vgg16
[INFO] DataLoader: Default Transform
[INFO] Number of epochs: 5


...

Epoch: 1 | train_loss: 0.1877 | train_acc: 0.9600 | test_loss: 0.0513 | test_acc: 0.9818
Epoch: 2 | train_loss: 0.0424 | train_acc: 0.9857 | test_loss: 0.0587 | test_acc: 0.9828
Epoch: 3 | train_loss: 0.0226 | train_acc: 0.9933 | test_loss: 0.1049 | test_acc: 0.9772
Epoch: 4 | train_loss: 0.0327 | train_acc: 0.9922 | test_loss: 0.1234 | test_acc: 0.9748
Epoch: 5 | train_loss: 0.0325 | train_acc: 0.9924 | test_loss: 0.1236 | test_acc: 0.9770
Epoch: 6 | train_loss: 0.0205 | train_acc: 0.9950 | test_loss: 0.1358 | test_acc: 0.9795
Epoch: 7 | train_loss: 0.0256 | train_acc: 0.9943 | test_loss: 0.1216 | test_acc: 0.9818
Epoch: 8 | train_loss: 0.0195 | train_acc: 0.9957 | test_loss: 0.1681 | test_acc: 0.9790
Epoch: 9 | train_loss: 0.0148 | train_acc: 0.9967 | test_loss: 0.1678 | test_acc: 0.9812
Epoch: 10 | train_loss: 0.0307 | train_acc: 0.9953 | test_loss: 0.2054 | test_acc: 0.9768
[INFO] Saving model to: models/vgg11_CLAHE + Rotation + Horizontal Flip Transform_10_epochs.pth
-------------

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.1706 | train_acc: 0.9700 | test_loss: 0.0567 | test_acc: 0.9858
Epoch: 2 | train_loss: 0.0368 | train_acc: 0.9895 | test_loss: 0.0546 | test_acc: 0.9870
Epoch: 3 | train_loss: 0.0143 | train_acc: 0.9956 | test_loss: 0.0935 | test_acc: 0.9835
Epoch: 4 | train_loss: 0.0170 | train_acc: 0.9953 | test_loss: 0.0901 | test_acc: 0.9842
Epoch: 5 | train_loss: 0.0160 | train_acc: 0.9962 | test_loss: 0.1149 | test_acc: 0.9855
Epoch: 6 | train_loss: 0.0252 | train_acc: 0.9951 | test_loss: 0.1486 | test_acc: 0.9850
Epoch: 7 | train_loss: 0.0239 | train_acc: 0.9957 | test_loss: 0.1799 | test_acc: 0.9818
Epoch: 8 | train_loss: 0.0573 | train_acc: 0.9936 | test_loss: 0.1817 | test_acc: 0.9858
Epoch: 9 | train_loss: 0.0456 | train_acc: 0.9957 | test_loss: 0.1749 | test_acc: 0.9868
Epoch: 10 | train_loss: 0.0142 | train_acc: 0.9980 | test_loss: 0.1887 | test_acc: 0.9868
[INFO] Saving model to: models/vgg16_CLAHE + Rotation + Horizontal Flip Transform_10_epochs.pth
-------------

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.1795 | train_acc: 0.9713 | test_loss: 0.0484 | test_acc: 0.9875
Epoch: 2 | train_loss: 0.0284 | train_acc: 0.9913 | test_loss: 0.0855 | test_acc: 0.9842
Epoch: 3 | train_loss: 0.0214 | train_acc: 0.9944 | test_loss: 0.0639 | test_acc: 0.9888
Epoch: 4 | train_loss: 0.0162 | train_acc: 0.9955 | test_loss: 0.0827 | test_acc: 0.9868
Epoch: 5 | train_loss: 0.0102 | train_acc: 0.9971 | test_loss: 0.0856 | test_acc: 0.9882
Epoch: 6 | train_loss: 0.0248 | train_acc: 0.9951 | test_loss: 0.1131 | test_acc: 0.9885
Epoch: 7 | train_loss: 0.0413 | train_acc: 0.9938 | test_loss: 0.1434 | test_acc: 0.9888
Epoch: 8 | train_loss: 0.0300 | train_acc: 0.9957 | test_loss: 0.2129 | test_acc: 0.9838
Epoch: 9 | train_loss: 0.0121 | train_acc: 0.9986 | test_loss: 0.1647 | test_acc: 0.9885
Epoch: 10 | train_loss: 0.0156 | train_acc: 0.9973 | test_loss: 0.2545 | test_acc: 0.9852
[INFO] Saving model to: models/vgg19_CLAHE + Rotation + Horizontal Flip Transform_10_epochs.pth
-------------